# Run inference on time to merge model trained previously


## What we did previously

In the previous [notebook](./03_model_training.ipynb) we trained machine learning models to classify a PR's `time_to_merge` into one of the 10 bins (or "classes"). We then deployed the model with the highest f1-score as a service using the model saved in s3.

## In this step

In this notebook, we are going to fetch the model that we previously trained, saved and stored in s3. We will send a payload to this model and see how it performs on the test data.
# Time to Merge Prediction Inference Service

In the previous notebook, we explored some basic machine learning models for predicting time to merge of a PR.

In [117]:
import os
import sys
import gzip
import json
import boto3
import datetime
import requests
from github import Github
from dotenv import load_dotenv, find_dotenv
from io import BytesIO

import joblib

import numpy as np
import pandas as pd

from sklearn.metrics import classification_report

load_dotenv(find_dotenv(), override=True)

import warnings
warnings.filterwarnings('ignore') 

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [118]:
# Recollect data for any new PR's
!python -m srcopsmetrics.cli --create-knowledge --repository $ORG/$REPO --entities PullRequest

15498.34s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


INFO:srcopsmetrics.github_knowledge:Overall repositories found: 1
INFO:srcopsmetrics.bot_knowledge:######################## Analysing MichaelClifford/ttmtool ########################

INFO:srcopsmetrics.bot_knowledge:########################
INFO:srcopsmetrics.bot_knowledge:Detected entities:
CodeFrequency # Commit # DependencyUpdate # Fork # Issue # IssueEvent # KebechetUpdateManager # License # PullRequest # PullRequestDiscussion # RawIssue # RawPullRequest # ReadMe # Release # Stargazer # ThothAdviseMetrics # ThothMetrics # ThothMetrics # ThothMetrics # ThothVersionManagerMetrics # TrafficClones # TrafficPaths # TrafficPaths # TrafficReferrers # TrafficClones # TrafficViews
INFO:srcopsmetrics.bot_knowledge:########################
INFO:srcopsmetrics.bot_knowledge:PullRequest inspection
INFO:srcopsmetrics.entities.tools.storage:Loading knowledge from Ceph
INFO:srcopsmetrics.entities.interface:Found previous PullRequest knowledge for MichaelClifford/ttmtool with 5 records
INFO:srcopsm

In [119]:
class CephCommunication:
    """
    Class to establish communication with a ceph s3 bucket.
    It connects with the bucket and provides methods to read and write data in the parquet format.
    """

    def __init__(
        self, s3_endpoint_url, aws_access_key_id, aws_secret_access_key, s3_bucket
    ):
        self.s3_endpoint_url = s3_endpoint_url
        self.aws_access_key_id = aws_access_key_id
        self.aws_secret_access_key = aws_secret_access_key
        self.s3_resource = boto3.resource(
            "s3",
            endpoint_url=self.s3_endpoint_url,
            aws_access_key_id=self.aws_access_key_id,
            aws_secret_access_key=self.aws_secret_access_key,
        )
        self.bucket = s3_bucket
        ## Todo: Add try catch

    def upload_to_ceph(self, dataframe, s3_path, filename):
        """
        This helper function takes as input the data frame to be uploaded, and the output filename.
        It then saves the data frame in the defined ceph bucket.
        """
        parquet_buffer = BytesIO()
        dataframe.to_parquet(parquet_buffer)
        s3_obj = self.s3_resource.Object(self.bucket, f"{s3_path}/{filename}")
        status = s3_obj.put(Body=parquet_buffer.getvalue())
        return status

    def read_from_ceph(self, s3_path, filename):
        """
        Helper function to read from ceph and see if the saved data is correct.
        """
        buffer = BytesIO()
        s3_object = self.s3_resource.Object(self.bucket, f"{s3_path}/{filename}")
        s3_object.download_fileobj(buffer)
        df_temp = pd.read_parquet(buffer)
        return df_temp


def save_to_disk(dataframe, path, filename):
    """
    Helper function to save the dataframe
    as a parquet file to disk.
    """
    dataset_base_path = Path(path)
    dataset_base_path.mkdir(parents=True, exist_ok=True)
    dataframe.to_parquet(f"{path}/{filename}")
    return True

In [120]:
## CEPH Bucket variables
## Create a .env file on your local with the correct configs,

ACTION = os.getenv("ACTION", 0)
ORG = os.getenv("GITHUB_ORG")
REPO = os.getenv("GITHUB_REPO")
TOKEN = os.getenv("GITHUB_ACCESS_TOKEN") 

## S3 bucket credentials
s3_endpoint_url = os.getenv("S3_ENDPOINT_URL")
s3_access_key = os.getenv("AWS_ACCESS_KEY_ID")
s3_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
s3_bucket = os.getenv("S3_BUCKET")

s3_input_data_path = os.getenv("CEPH_BUCKET_PREFIX")

REMOTE = os.getenv("REMOTE")
RAW_DATA_PATH = os.path.join(
    s3_input_data_path, "srcopsmetrics/bot_knowledge", ORG, REPO, "PullRequest.json"
)

In [121]:
output = []
local_input_data_path = "../../../data/raw/GitHub/PullRequest.json.gz"
if REMOTE:
    print("getting dataset from ceph")
    s3 = boto3.resource(
        "s3",
        endpoint_url=s3_endpoint_url,
        aws_access_key_id=s3_access_key,
        aws_secret_access_key=s3_secret_key,
    )
    content = s3.Object(s3_bucket, RAW_DATA_PATH)
    file = content.get()["Body"].read().decode("utf-8")

    prs = json.loads(file)

    for pr in prs.splitlines():
        output.append(json.loads(pr))

else:
    print("getting dataset from local")
    with gzip.open(local_input_data_path, "r") as f:
        prs = json.loads(f.read().decode("utf-8"))


pr_df = pd.DataFrame(output)

getting dataset from ceph


In [122]:
# github pr dataset collected using thoth's mi-scheduler
pr_df.head()

,body,changed_files,changed_files_number,closed_at,closed_by,commits,commits_number,created_at,created_by,first_approve_at,first_review_at,interactions,labels,merged_at,merged_by,reviews,size,title,id
0,None,"[.github/workflows/infer.yaml, 01_notebook_exe...",6,NaN,None,"[54c204441eff0f5ec146d9938a72247ed622a148, 9f3...",11,1667954112,MichaelClifford,None,NaN,{'review-notebook-app[bot]': 29},[],NaN,None,{},XXL,One,5
1,None,"[.github/workflows/infer.yaml, 01_notebook_exe...",6,NaN,None,"[54c204441eff0f5ec146d9938a72247ed622a148, 9f3...",12,1667603542000,MichaelClifford,None,NaN,"{'github-actions[bot]': 41, 'MichaelClifford': 2}",[],NaN,None,{},XXL,two,4
2,None,[.github/workflows/infer.yaml],1,1.667603e+12,MichaelClifford,[db14b45a3a47af40e95bad4cb4904e01badf7411],1,1667602887000,MichaelClifford,None,NaN,{'MichaelClifford': 2},[],1.667603e+12,MichaelClifford,{},XS,3,3
3,None,"[.github/workflows/infer.yaml, changingfile.txt]",2,1.667602e+12,MichaelClifford,"[ecbb271c622803a69ac1eb30c28ffdcf9ea6c9d0, 2ce...",2,1667602300000,MichaelClifford,None,NaN,{},[],1.667602e+12,MichaelClifford,{},S,One,2
4,None,"[.github/workflows/infer.yaml, changingfile.txt]",2,1.667602e+12,MichaelClifford,"[ecbb271c622803a69ac1eb30c28ffdcf9ea6c9d0, 2ce...",2,1667592663000,MichaelClifford,None,1.667593e+12,{},[],NaN,None,{'1168831499': {'author': 'github-actions[bot]...,S,one,1


In [123]:
interval = (pr_df["merged_at"] - pr_df["created_at"]).astype("float")
interval = interval.dropna()
interval

2    549000.0
3     72000.0
dtype: float64

In [124]:
n_buckets = 10

quantiles = interval.quantile(q=np.arange(0, 1 + 1e-100, 1 / n_buckets))
quantiles

0.0     72000.0
0.1    119700.0
0.2    167400.0
0.3    215100.0
0.4    262800.0
0.5    310500.0
0.6    358200.0
0.7    405900.0
0.8    453600.0
0.9    501300.0
dtype: float64

In [125]:
quantiles / 3600

0.0     20.00
0.1     33.25
0.2     46.50
0.3     59.75
0.4     73.00
0.5     86.25
0.6     99.50
0.7    112.75
0.8    126.00
0.9    139.25
dtype: float64

In [126]:
time_intervals = quantiles / 3600

In [127]:
# remove PRs from train/test which are still open
pr_df = pr_df[pr_df["closed_at"].notna()]
pr_df = pr_df[pr_df["merged_at"].notna()]

In [128]:
pr_df["created_at"] = pr_df["created_at"].apply(
    lambda x: int(datetime.datetime.timestamp(pd.to_datetime(x)))
)
pr_df["closed_at"] = pr_df["closed_at"].apply(
    lambda x: float(datetime.datetime.timestamp(pd.to_datetime(x)))
)
pr_df["merged_at"] = pr_df["merged_at"].apply(
    lambda x: float(datetime.datetime.timestamp(pd.to_datetime(x)))
)

In [129]:
TEST_DATA_PATH = os.path.join(s3_input_data_path, ORG, REPO, "test-data")

# read processed and split data created for train/test in the model training notebook
if REMOTE:
    cc = CephCommunication(s3_endpoint_url, s3_access_key, s3_secret_key, s3_bucket)
    X_test = cc.read_from_ceph(TEST_DATA_PATH, "X_test.parquet")
    y_test = cc.read_from_ceph(TEST_DATA_PATH, "y_test.parquet")

else:
    print(
        "The X_test.parquet and y_test.parquet files are not included in the github repo."
    )
    print(
        "Please set REMOTE=1 in the .env file and read this data from the S3 bucket instead."
    )

In [130]:
X_test.head()

,size,created_at_day,created_at_month,created_at_weekday,created_at_hour,changed_files_number,body_size,commits_number,filetype_.ipynb,filetype_.txt,title_wordcount_one,title_wordcount_two
2,1.0,4.0,11.0,4.0,18.0,2.0,0.0,2.0,0.0,1.0,1.0,0.0


In [131]:
y_test.head()

,ttm_class
2,0


In [132]:
sample_payload = pr_df.reindex(X_test.index)

In [133]:
sample_payload.head()

,body,changed_files,changed_files_number,closed_at,closed_by,commits,commits_number,created_at,created_by,first_approve_at,first_review_at,interactions,labels,merged_at,merged_by,reviews,size,title,id
2,None,[.github/workflows/infer.yaml],1,19667.603436,MichaelClifford,[db14b45a3a47af40e95bad4cb4904e01badf7411],1,19667,MichaelClifford,None,NaN,{'MichaelClifford': 2},[],19667.603436,MichaelClifford,{},XS,3,3


In [135]:
## read model
MODEL_KEY = os.path.join(s3_input_data_path, ORG, REPO, "ttm-model")
MODEL_FILENAME = "model.joblib"


s3_resource = boto3.resource(
    "s3",
    endpoint_url=s3_endpoint_url,
    aws_access_key_id=s3_access_key,
    aws_secret_access_key=s3_secret_key,
)

buffer = BytesIO()
s3_object = s3_resource.Object(s3_bucket, f"{MODEL_KEY}/{MODEL_FILENAME}")
s3_object.download_fileobj(buffer)
model = joblib.load(buffer)
model

Pipeline(steps=[('extract_features',
                 ColumnTransformer(transformers=[('pr_size',
                                                  OrdinalEncoder(categories=[['XS',
                                                                              'S',
                                                                              'M',
                                                                              'L',
                                                                              'XL',
                                                                              'XXL']]),
                                                  ['size']),
                                                 ('created_at_details',
                                                  DateTimeDetailsTransformer(),
                                                  ['created_at']),
                                                 ('n_changed_files',
                                                  NumChangedFilesTransformer(),
                                                  ['changed_files_number']),
                                                 ('body_size',
                                                  StringLenTransformer(),
                                                  ['body']),
                                                 ('n_commits', 'passthrough',
                                                  ['commits_number']),
                                                 ('filetype_counter',
                                                  FileTypeCountTransformer(file_extensions=['.ipynb',
                                                                                            '.txt']),
                                                  ['changed_files']),
                                                 ('title_word_counter',
                                                  TitleWordCountTransformer(words=['one',
                                                                                   'two']),
                                                  ['title'])])),
                ('scale', PowerTransformer()), ('rf', GaussianNB())])

In [136]:
# Test model on the dataset
preds = model.predict(sample_payload)
cr = classification_report(y_test, preds)
print(cr)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           9       0.00      0.00      0.00       0.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0



In [138]:
if ACTION:
    new_pr = sample_payload.iloc[-1:]
    prediction = model.predict(new_pr) 
    g = Github(TOKEN)
    repo = g.get_repo(f"{ORG}/{REPO}")
    prs = repo.get_pulls(state='all', sort='created', base='main')
    pr_nums = [pr.number for pr in prs]
    pr = repo.get_pull(pr_nums[-1])
    pr.create_issue_comment(f"Our Model Predicts this PR to be in category {prediction}")

# Conclusion

This notebook fetches the saved model from s3 and sends a payload to see how the model is performing on this new data. Additionally, we see that the evaluation scores in the classification report match the ones we saw in the training notebook. So, great, looks like our model are working as expected, and are ready to predict some times to merge for GitHub PRs! 